In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
#from folium.element import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
#import pysal as ps
from IPython.display import IFrame
import jenkspy
import seaborn as sns
import matplotlib.pyplot as plt

def add_choropleth(mapobj, gdf, id_field, value_field,  fill_color='YlOrRd', fill_opacity=0.2, line_opacity=0.001, 
                     num_classes =10, classifier = 'Fisher_Jenks'):
    print(fill_color)
    threshold_scale= jenkspy.jenks_breaks(gdf[value_field], nb_class=5)
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    print(threshold_scale)
     
    mapobj.choropleth(geo_str = gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
           
    return mapobj
def add_point_clusters(mapobj, gdf, popup_field_list):
    coords, popups = [], [] 

    marker_cluster = folium.MarkerCluster()
    colorpalet=sns.color_palette("YlOrRd", 5)
    threshold_scale= jenkspy.jenks_breaks(gdf["number_of_violations"], nb_class=5)

    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = 'Number of Violations : '+" ".join([str(row[field]) for field in popup_field_list])
        popups.append(label)#IFrame(label, width = 400, height = 500))
        #
        
        #row["number_of_violations"]
        circlemarker1=folium.CircleMarker([row.geometry.y, row.geometry.x], popup=label, color='#f33c25',fill_color='#ff0000',  radius=5).add_to(marker_cluster)
   


    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    #pt_lyr.add_child(MarkerCluster(locations = coords, popups = popups))
    pt_lyr.add_child(marker_cluster)
    #circlemarker=folium.CircleMarker(location = coords,radius=300,color='#3186cc',fill_color='#3186cc')
    
    #Add this point layer to the map object
    mapobj.add_child(pt_lyr)

    return mapobj

violationdf=pd.read_csv('allcoordinate.csv',keep_default_na=False)
#violationdf=violationdf[(violationdf.cluster==0) | (violationdf.cluster==1)| (violationdf.cluster==4)| (violationdf.cluster==5)]
#violationdf.to_csv('/Users/ingridwijaya/Desktop/Practicum/allcoordinatewithoutnoise.csv')
violoation_geo = gpd.GeoSeries(violationdf.apply(lambda z: Point(z['longitude'], z['latitude']), 1),crs={'init': 'epsg:4326'})
violationdf = gpd.GeoDataFrame(violationdf.drop(['latitude', 'longitude'], 1), geometry=violoation_geo)
#BUSH C40.5212, -74.4623
#40.4793, -74.4332
#40.4982° N, 74.4468° W
#40.4982, -74.4468
#40.5240° N, 74.4355° W
#40.5240, -74.4355 
#40.5008° N, 74.4474° W
violation_map = folium.Map([40.5008, -74.4474], zoom_start = 13)
violationshape = gpd.read_file('allcoordinate_grid2.shp',crs={'init': 'epsg:4326'})
violationshape['ViolationPSqMi'] =violationshape.NumberOfVi#/(violationshape.geometry.area*3.86102e-7)).fillna(0)
violation_map=add_choropleth(violation_map, violationshape, 'ID','ViolationPSqMi')
violation_map = add_point_clusters(violation_map, violationdf, ['number_of_violations'])
folium.LayerControl().add_to(violation_map)
violation_map.save('allviolation_map.html')



violation_map
#sns.palplot(sns.color_palette("Blues", 5))

YlOrRd
[0.0, 10.0, 41.0, 89.0, 162.0, 356.0]
